In [1]:
import time
import cv2
import glob
import os
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.svm import LinearSVC

print(cv2.__version__)

4.5.2


In [ ]:
start_time = time.time()

'''
Parameters
'''
patch_stride = 16
K = 20



'''
Normalizing
'''
flattened_train_features = np.concatenate(train_features, axis=0)
pca = PCA(n_components=flattened_train_features.shape[-1], whiten=True)
pca.fit(flattened_train_features)
train_normalized_features = list()
index = 0
for features in train_features:
    features = pca.transform(features)
    train_normalized_features.append(features)
    index += 1
    print("Normalize Train Features ... {:4d}/{:4d}".format(index, len(train_features)))
test_normalized_features = list()
index = 0
for features in test_features:
    features = pca.transform(features)
    test_normalized_features.append(features)
    index += 1
    print("Normalize Test Features ... {:4d}/{:4d}".format(index, len(test_features)))

'''
Make Codebook
'''
###################################################################################
# 아래의 코드의 빈 곳(None 부분)을 채우세요.
# None 부분 외의 부분은 가급적 수정 하지 말고, 주어진 형식에 맞추어
# None 부분 만을 채워주세요. 임의적으로 전체적인 구조를 수정하셔도 좋지만,
# 파이썬 코딩에 익숙 하지 않으시면, 가급적 틀을 유지하시는 것을 권장합니다.
# 1) 함수 encode 부분 안의 None 부분을 채우세요.
#    distances는 K means 알고리즘을 통해 얻어진 centroids, 즉 codewords(visual words)와
#    각 이미지의 특징들 간의 거리 입니다.
#    distances 값을 이용하여, features(# of keypoints, feature_dim)를
#    인코딩(histogram 혹은 quantization이라고도 함) 하세요.
#    인코딩된 결과인 representations은 (K)로 표현되어야 합니다.
#    이 때, K는 codewords의 개수입니다.
###################################################################################
class Codebook:

    def __init__(self, K):

        self.K = K

        self.kmeans = KMeans(n_clusters=K, verbose=True)

    def make_code_words(self, features):

        self.kmeans.fit(features)

    def encode(self, features, shapes):

        distances = self.kmeans.transform(features)

        # reshaped_features는 Spatial Pyramid Matching 문제에 활용하세요.
        reshaped_features = np.reshape(features, (shapes[0], shapes[1], -1))

        # Write Your Code Here ########################################################
        representations = None
        ###############################################################################

        if np.array(representations).shape != (self.K, ):
            # representations는 반드시 (K) 차원을 가져야 합니다 (Spatial Pyramid Matching 사용 안할 시에만).
            print("Your code may be wrong")

        return representations

print("Make Codebook ...")
flattened_normalized_train_features = pca.transform(flattened_train_features)
codebook = Codebook(K)
codebook.make_code_words(flattened_normalized_train_features)

'''
Encode Features
'''
train_encoded_features = list()
index = 0
for features, shapes in zip(train_normalized_features, train_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    train_encoded_features.append(encoded_features)
    index += 1
    print("Encoding Train Features ... {:4d}/{:4d}".format(index, len(train_normalized_features)))
test_encoded_features = list()
index = 0
for features, shapes in zip(test_normalized_features, test_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    test_encoded_features.append(encoded_features)
    index += 1
    print("Encoding Text Features ... {:4d}/{:4d}".format(index, len(test_normalized_features)))

'''
Approximate Kernel
'''
chi2sampler = AdditiveChi2Sampler(sample_steps=2)
chi2sampler.fit(train_encoded_features, y_train)
train_encoded_features = chi2sampler.transform(train_encoded_features)
test_encoded_features = chi2sampler.transform(test_encoded_features)

'''
Classify Images with SVM
'''
###################################################################################
# 아래의 코드의 빈 곳(None 부분)을 채우세요.
# None 부분 외의 부분은 가급적 수정 하지 말고, 주어진 형식에 맞추어
# None 부분 만을 채워주세요. 임의적으로 전체적인 구조를 수정하셔도 좋지만,
# 파이썬 코딩에 익숙 하지 않으시면, 가급적 틀을 유지하시는 것을 권장합니다.
# 1) 아래의 model 부분에 sklearn 패키지를 활용하여, Linear SVM(SVC) 모델을 정의하세요.
#    처음에는 SVM의 parameter를 기본으로 설정하여 구동하시길 권장합니다.
#    구동 성공 시, SVM의 C 값과 max_iter 파라미터 등을 조정하여 성능 향상을 해보시길 바랍니다.
###################################################################################
# Write Your Code Here ############################################################
model = None
###################################################################################

print("Classify Images ...")
model.fit(train_encoded_features, y_train)
train_score = model.score(train_encoded_features, y_train)
test_score = model.score(test_encoded_features, y_test)

elapsed_time = time.time() - start_time

'''
Print Results
'''
print()
print("=" * 90)
print("Train  Score: {:.5f}".format(train_score))
print("Test   Score: {:.5f}".format(test_score))
print("Elapsed Time: {:.2f} secs".format(elapsed_time))
print("=" * 90)

[문제 1. 특징 추출 – 15 pts]  
1-1. 주어진 Keypoint를 통하여, 각각의 이미지마다 특징을 추출하시오.  
이 때, OpenCV 패키지를 활용하여, SIFT와 같은 특징을 추출하시오. (자세한 사항은 템플릿 코드를 참고)  

In [ ]:
# 이 keypoint로 추출한다는걸 잘 모르겠다.

In [ ]:
'''
Load Dataset
'''
def scene15():

    train_folders = glob.glob("SCENE-15/train/*")
    train_folders.sort()
    classes = dict()
    x_train = list()
    y_train = list()
    for index, folder in enumerate(train_folders):
        label = os.path.basename(folder) #머지
        classes[label] = index
        paths = glob.glob(os.path.join(folder, "*"))
        for path in paths:
            x_train.append(cv2.imread(path, 0))
            y_train.append(index)

    x_test = list()
    y_test = list()
    test_folders = glob.glob("SCENE-15/test/*")
    test_folders.sort()
    for folder in test_folders:
        label = os.path.basename(folder)
        index = classes[label]
        paths = glob.glob(os.path.join(folder, "*"))
        for path in paths:
            x_test.append(cv2.imread(path, 0))
            y_test.append(index)

    return x_train, y_train, x_test, y_test, sorted(classes.keys())

print("Load Dataset ...")
x_train, y_train, x_test, y_test, labels_names = scene15()

random_indices = list(range(len(y_train)))
random.shuffle(random_indices)
x_train = np.array(x_train)[random_indices].tolist()
y_train = np.array(y_train)[random_indices].tolist()

In [ ]:

'''
Extract Patches
'''
train_key_points = list()
train_feature_shapes = list()
for image in x_train:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    train_key_points.append(image_key_points)
    train_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

test_key_points = list()
test_feature_shapes = list()
for image in x_test:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    test_key_points.append(image_key_points)
    test_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))


[문제 2. Bag-of-Features 구현 – 30 pts]  
2-1. K-means을 통해 구해진 codebook을 통하여, 각각의 이미지의 특징을 인코딩(histogram화 혹은  양자화 라고도 함) 하시오.  
(자세한 사항은 템플릿 코드를 참고)  

In [ ]:
'''
Extract Features
'''
###################################################################################
# 아래의 코드의 빈 곳(None 부분)을 채우세요.
# None 부분 외의 부분은 가급적 수정 하지 말고, 주어진 형식에 맞추어
# None 부분 만을 채워주세요. 임의적으로 전체적인 구조를 수정하셔도 좋지만,
# 파이썬 코딩에 익숙 하지 않으시면, 가급적 틀을 유지하시는 것을 권장합니다.
# 1) descriptor를 선정하세요. (SIFT, SURF 등) OpenCV의 패키지를 사용하시면 됩니다.
# 2) for 반복문 안에서, 1)에서 정의한 descriptor를 통하여 features를 추출하세요.
#    features의 차원은 (# of keypoints, feature_dim) 입니다.
###################################################################################
# Write Your Code Here ############################################################
descriptor = None
###################################################################################
train_features = list()
index = 0
for image, key_points in zip(x_train, train_key_points):
    # Write Your Code Here ########################################################
    _, features = None
    ###############################################################################
    train_features.append(features)
    index += 1
    print("Extract Train Features ... {:4d}/{:4d}".format(index, len(x_train)))

test_features = list()
index = 0
for image, key_points in zip(x_test, test_key_points):
    # Write Your Code Here ########################################################
    _, features = None
    ###############################################################################
    test_features.append(features)
    index += 1
    print("Extract Test Features ... {:4d}/{:4d}".format(index, len(x_test)))


[문제 3. SVM을 통한 이미지 분류 – 15 pts]  
3-1. Bag-of-Features 알고리즘을 통해 얻어진 인코딩된 벡터를 통해, SVM을 학습하시오.  
이 때, sklearn과 같은 패키지를 활용하여 SVM 학습을 구현하시면 됩니다. (자세한 사항은 템플릿 코드를 참고)

[문제 4. 파라미터 조정을 통한 성능 개선 – 40 pts]  
4-1. 현재 주어진 코드에는 여러 하이퍼 파라미터가 존재합니다.    
예를 들어, Keypoint를 만들 때의 간격(patch_stride), codebook의 visual word 개수(K), SVM의 학습 파라미터 (C 값 및 max_iter) 등 이 있습니다.   
해당 파라미터들을 수정해가면서 결과를 개선시켜 보시오. (K와 patch_stride 같은 경우, 메모리 부족으로 큰 데이터를 사용하지 못 할 수 있으므로, 절대적인 성능 지표보다, 상대적인 성능 향상을 더 고려할 예정임)  

4-2. 앞서 여러 파라미터의 실험 결과를 토대로 개선된 혹은 개선 되지 않은 이유를 보고서에 설명하시오. (예를 들어, 각 파라미터 별로 결과 테이블 정리 및 결과에 대한 이유 등).

[심화 문제 5. 알고리즘 개선 – 가산점 +20 pts]  
(본 문제는 풀이를 하지 않으셔도 감점되지 않습니다.)  
5-1. Bag-of-Features 알고리즘의 단점 중 하나는 공간의 배열, 위치 관계를 보지 않고 특징을 인코딩한다는 점입니다.   
즉, 특정 특징들의 개수 만을 통해서 인코딩을 하기 때문에 물체의 위치, 배열 등의 패턴 차이는 제대로 표현되지 않을 수 있습니다.   
이러한 단점을 극복하기 위하여 Spatial Pyramid Matching (공간 분할)을 수행 후, codewords을 통해 인코딩 할 수 있습니다.   
앞서 언급된 Spatial Pyramid Matching 기법을 적용하여 성능을 개선 시켜 보시오.  
(https://slazebni.cs.illinois.edu/publications/pyramid_chapter.pdf를 참고하시면 도움이 됩니다.)